In [16]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import datetime
import tensorflow as tf
import itertools
import matplotlib
from matplotlib import pyplot
import matplotlib.pyplot as plt
from sklearn.utils import class_weight
import tensorflow_addons as tfa
from sklearn.model_selection import StratifiedKFold
import math
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
import plotly.express as px
from plotly.subplots import make_subplots
import seaborn as sns
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")
from IPython.display import clear_output
from sklearn.metrics import log_loss
import xgboost as xgb
import random
import optuna
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split,StratifiedKFold
from catboost import CatBoostClassifier

In [2]:
df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [3]:
titles1 = []
titles2 = []
marital1 = []
marital2 = []

for i in df['Name']:
    titles1.append(i.split(',')[0])
    marital1.append(i.split('.')[0].split(',')[1])
    
for i in test_df['Name']:
    titles2.append(i.split(',')[0])
    marital2.append(i.split('.')[0].split(',')[1])

In [4]:
df['Titlles'] = titles1
df['marital_status'] = marital1
test_df['Titles'] = titles2
test_df['marital_status'] = marital2
a = df['marital_status'].value_counts()
a

 Mr              517
 Miss            182
 Mrs             125
 Master           40
 Dr                7
 Rev               6
 Major             2
 Mlle              2
 Col               2
 the Countess      1
 Lady              1
 Mme               1
 Ms                1
 Jonkheer          1
 Capt              1
 Don               1
 Sir               1
Name: marital_status, dtype: int64

In [5]:
cleanup_nums = {"marital_status":{" Mr": 1, " Mrs":1, " Lady": 1, " Miss":0, " Master":0}}#mr + mrs + lady, miss + master
df = df.replace(cleanup_nums)
a = df['marital_status'].value_counts()
a

1                643
0                222
 Dr                7
 Rev               6
 Major             2
 Mlle              2
 Col               2
 Jonkheer          1
 the Countess      1
 Mme               1
 Ms                1
 Capt              1
 Don               1
 Sir               1
Name: marital_status, dtype: int64

In [6]:
test_df = test_df.replace(cleanup_nums)
df.loc[(df.marital_status.isin([" Dr", " Rev", " Mlle", " Major", " Col", " Mme", " Don", " Capt", " Jonkheer", " Ms", " the Countess", " Sir"])) & (df['SibSp']>0) , 'marital_status'] = 1
df.loc[(df.marital_status.isin([" Dr", " Rev", " Mlle", " Major", " Col", " Mme", " Don", " Capt", " Jonkheer", " Ms", " the Countess", " Sir"])) & (df['SibSp']==0) , 'marital_status'] = 0
test_df.loc[(test_df.marital_status.isin([" Dr", " Rev", " Mlle", " Major", " Col", " Mme", " Don", " Capt", " Jonkheer", " Ms", " the Countess", " Sir", " Dona"])) & (test_df['SibSp']>0) , 'marital_status'] = 1
test_df.loc[(test_df.marital_status.isin([" Dr", " Rev", " Mlle", " Major", " Col", " Mme", " Don", " Capt", " Jonkheer", " Ms", " the Countess", " Sir", " Dona"])) & (test_df['SibSp']==0) , 'marital_status'] = 0

In [7]:
df['marital_status'] = df['marital_status'].astype(int)
test_df['marital_status'] = test_df['marital_status'].astype(int)

In [8]:
%%capture
df['survived']=df['Sex']
mask=(df['Survived']==1)
df['survived'][mask]='survived'
mask=(df['Survived']==0)
df['survived'][mask]='died'

In [19]:
#dropping unnecesary columns NOTE survived was a column created to aid visualization
# df=df.drop(['PassengerId', 'Name', 'survived', 'Ticket','relatives'], axis=1)
#filling Embarked with most frequent in the column
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].value_counts().index[0])
df['Embarked'].value_counts()

3    646
2    168
1     77
Name: Embarked, dtype: int64

In [20]:
#replacing manually the categorical variable other encoders could be used
cleanup_nums = {"Embarked":     {"Q": 1, "C":2, "S":3},
                "Sex": {"male": 0, "female": 1}
               }
df = df.replace(cleanup_nums)

In [21]:
#total no of relatives for a person on the ship
df['family']=df['SibSp']+df['Parch']+1

In [25]:
#its not realistic for fare to be zero which is special case so let us replace with mean
df['Fare'] = df['Fare'].replace(0, df['Fare'].mean())

In [13]:
#if has a cabin then 1 else 0
df['Cabin']=df['Cabin'].fillna("0")
df.loc[df.Cabin != "0", 'Cabin'] = "1"
df['Cabin']=df['Cabin'].astype(int)

In [17]:
#if has no relative then 0 else 1
df['isAlone']=np.zeros(len(df))
df.loc[df.family == 1, 'isAlone'] = 1
dp=df.dropna()

In [37]:
#we will not use survived as a feature to predict the age to fill the missing ages as it will not be available in test dataset
X=dp.drop(['Age', 'Titles'], axis=1)
y=dp.Age

In [38]:
%%capture
test_df=test_df.drop(['PassengerId', 'Name', 'Ticket'], axis=1)

test_df['family']=test_df['SibSp']+test_df['Parch']+1

test_df['Embarked']=test_df['Embarked'].fillna('G')
test_df = test_df.replace(cleanup_nums)
test_df['Fare'] = test_df['Fare'].replace(0, test_df['Fare'].mean())
test_df['Fare']=test_df['Fare'].fillna(test_df['Fare'].mean())
test_df['Cabin']=test_df['Cabin'].fillna("0")
test_df.loc[df.Cabin != "0", 'Cabin'] = "1"
test_df['Cabin']=test_df['Cabin'].astype(int)
test_df['isAlone']=np.zeros(len(test_df))
test_df.loc[test_df.family == 1, 'isAlone'] = 1
dp=test_df.dropna()
x=dp.drop(['Age','Titles'], axis=1)
Y=dp.Age

KeyError: "['PassengerId' 'Name' 'Ticket'] not found in axis"

In [39]:
X=pd.concat([X, x])
y=pd.concat([y, Y])
X.reset_index()
y.reset_index()
print(X.shape)
print(y.shape)

(664, 10)
(664,)


In [40]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Titlles,marital_status,survived,family,isAlone
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,0,3,Braund,1,died,2,0.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,1,2,Cumings,1,survived,2,0.0
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,0,3,Heikkinen,0,survived,1,1.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,1,3,Futrelle,1,survived,2,0.0
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,0,3,Allen,1,died,1,1.0


**Save to csv file**

In [43]:
X.to_csv('train.csv',index=False)
y.to_csv('test.csv', index=False)